<a href="https://colab.research.google.com/github/olfabre/amsProjetMaster1/blob/olivier/ShakeSpeare_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Version initiale

In [5]:
try:
    import unidecode
except ModuleNotFoundError:
    !pip install unidecode
    import unidecode
import string
import random
import re
import os
import requests

import torch
import torch.nn as nn
from torch.autograd import Variable

import time
import math
import matplotlib.pyplot as plt
from argparse import ArgumentParser

# Vérification du GPU
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("CUDA AVAILABLE")
else:
    device = torch.device("cpu")
    print("ONLY CPU AVAILABLE")

# Paramètres globaux
all_characters = string.printable
n_characters = len(all_characters)
chunk_len = 13

n_epochs = 200000
print_every = 10
plot_every = 10
hidden_size = 512
n_layers = 3
lr = 0.005

# Téléchargement des données depuis une URL
def download_data(url, filename):
    response = requests.get(url)
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(response.text)

# Chargement des données
url = "https://olivier-fabre.com/passwordgenius/shakespeare2.txt"
data_dir = "data"
os.makedirs(data_dir, exist_ok=True)
data_path = os.path.join(data_dir, "shakespeare2.txt")

if not os.path.exists(data_path):
    print("Téléchargement des données...")
    download_data(url, data_path)

# Lecture et traitement du fichier
file = unidecode.unidecode(open(data_path, "r", encoding="utf-8").read())
file_len = len(file)
print(f"Longueur du corpus : {file_len}")

# Fonctions de préparation des données
def random_chunk(file):
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

def random_training_set(file):
    chunk = random_chunk(file)
    inp = char_tensor(chunk[:-1]).to(device)
    target = char_tensor(chunk[1:]).to(device)
    return inp, target

# Définition du modèle
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size, device=device))

# Fonctions d'entraînement et d'évaluation
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0
    for c in range(inp.size(0)):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c].unsqueeze(0))
    loss.backward()
    decoder_optimizer.step()
    return loss.item() / chunk_len

def training(n_epochs, file):
    start = time.time()
    for epoch in range(1, n_epochs + 1):
        loss = train(*random_training_set(file))
        if epoch % print_every == 0:
            print(f"[{time_since(start)} ({epoch}/{n_epochs})] Perte : {loss:.4f}")

def evaluate(decoder, prime_str="A", predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str).to(device)
    predicted = prime_str
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char).to(device)
    return predicted

def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return f'{m}m {s:.2f}s'

# Lancement principal
if __name__ == "__main__":
    decoder = RNN(n_characters, hidden_size, n_characters, n_layers).to(device)
    decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    print("Début de l'entraînement...")
    training(1000, file)  # Ajustez n_epochs pour vos besoins

    print("\nÉvaluation...")
    print(evaluate(decoder, prime_str="To be or not to be", predict_len=200, temperature=0.8))


CUDA AVAILABLE
Longueur du corpus : 314122
Début de l'entraînement...
 0.37s (10/1000)] Perte : 4.1916
 0.75s (20/1000)] Perte : 6.0787
 1.11s (30/1000)] Perte : 4.0047
 1.48s (40/1000)] Perte : 3.0178
 1.89s (50/1000)] Perte : 3.6313
 2.28s (60/1000)] Perte : 3.4829
 2.55s (70/1000)] Perte : 3.4531
 2.80s (80/1000)] Perte : 3.3117
 3.05s (90/1000)] Perte : 3.5916
 3.29s (100/1000)] Perte : 4.2990
 3.52s (110/1000)] Perte : 3.5007
 3.78s (120/1000)] Perte : 2.8189
 4.02s (130/1000)] Perte : 4.8903
 4.25s (140/1000)] Perte : 3.2798
 4.49s (150/1000)] Perte : 4.0331
 4.72s (160/1000)] Perte : 3.0311
 4.97s (170/1000)] Perte : 3.0005
 5.19s (180/1000)] Perte : 3.1937
 5.43s (190/1000)] Perte : 3.1834
 5.68s (200/1000)] Perte : 3.1069
 5.93s (210/1000)] Perte : 2.8318
 6.17s (220/1000)] Perte : 2.9983
 6.41s (230/1000)] Perte : 3.0574
 6.64s (240/1000)] Perte : 3.4660
 6.89s (250/1000)] Perte : 3.8896
 7.13s (260/1000)] Perte : 2.7422
 7.37s (270/1000)] Perte : 2.9514
 7.60s (280/1000)] Pe